In [ ]:
import utils
import feature_extractor
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import pickle

# Constant Definition

In [ ]:
NUM_CLASSES=23
BATCH_SIZE=64
EPOCHS=10
HIDDEN_SIZE=64
VECTOR_SIZE = 200  # Size of word vectors
WINDOW_SIZE = 5  # Context window size
THREADS = 4  # Number of threads to use for training
CUTOFF_FREQ = 1  # Minimum frequency for a word to be included in vocabulary
TRAINING_SIZE = 10000  
TEST_SIZE = 10

# reading dataset

In [ ]:
complete_data=utils.read_data("../data/fixed_PIZZA_train.json")

In [ ]:
data = complete_data[:TRAINING_SIZE]
corpus, top, decoupled = utils.get_train_dataset(data)

In [ ]:
entites_output_as_number_labels,intents_output_as_number_labels, input_as_tokenized_string=utils.label_complete_input(corpus, decoupled, top)

# Embedding model

In [ ]:
reviews_tokens = []
with open('../data/food_review.txt', 'r', encoding='utf-8') as f:
    for line in f:
        reviews_tokens.append(utils.tokenize_string(line.strip()))
print("Done1")
with open('../data/food_review2.txt', 'r', encoding='utf-8') as f:
    for line in f:
        reviews_tokens.append(utils.tokenize_string(line.strip()))
print("Done2")
all_trainig = []
all_corpus, _, _ = utils.get_train_dataset(complete_data)
for i in range(len(all_corpus)):
    all_trainig.append(utils.tokenize_string(all_corpus[i])) 
print("Done3")
emb_model = feature_extractor.train_gensim_w2v_model(all_trainig+reviews_tokens, VECTOR_SIZE)

In [ ]:
from gensim.models import Word2Vec, FastText  # For Word2Vec model
emb_model.save('../embedding_models/word2vec_whole.model')
# emb_model = Word2Vec.load('../models/word2vec_food_reviews.model')

In [ ]:
print(len(all_trainig+reviews_tokens), +10000+2456446)

In [ ]:
emb_model.wv.most_similar('coca-cola')

# NER Model

In [ ]:
class LargeDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data 
        self.labels = labels 
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

def collate_fn(batch):
    sequences, labels = zip(*batch)
    #I believe we can transform words into embeddings here
    embeddings=[]
    # print(sequences)
    for seq in sequences:
        x=[]
        for token in seq:
            x.append(emb_model.wv[token])
        embeddings.append(x)
    sequences=embeddings
    labels = [torch.tensor(label, dtype=torch.long) for label in labels]
    padded_labels = pad_sequence(labels, batch_first=True, padding_value=-1)
    sequences = [torch.tensor(seq) for seq in sequences]
    padded_sequences = pad_sequence(sequences, batch_first=True)
    lengths = torch.tensor([len(seq) for seq in sequences], dtype=torch.long)
    return padded_sequences, padded_labels, lengths

class LargeWordRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LargeWordRNN, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, num_classes)
    
    def forward(self, x, lengths):
        packed_x = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_out, _ = self.rnn(packed_x)
        out, _ = pad_packed_sequence(packed_out, batch_first=True)
        out = self.fc(out)
        return out

# Checking if CUDA is working

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available()) 
print(torch.version.cuda)
print(f'Using device: {device}')

# Training Model

In [ ]:


labels = entites_output_as_number_labels

dataset = LargeDataset(input_as_tokenized_string, labels)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=True, num_workers=0)

model = LargeWordRNN(input_size=VECTOR_SIZE, hidden_size=HIDDEN_SIZE, num_classes=NUM_CLASSES).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=-1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
for epoch in range(EPOCHS): 
    for padded_sequences, padded_labels, lengths in dataloader:
        padded_sequences=padded_sequences.to(device)
        padded_labels=padded_labels.to(device)
        lengths=lengths.to(device)
        optimizer.zero_grad()
        outputs = model(padded_sequences, lengths)
        loss = criterion(outputs.view(-1, NUM_CLASSES), padded_labels.view(-1))
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch + 1}, Loss: {loss.item():.4f}")


In [ ]:
# test_data=complete_data[200000:200010]
# print(test_data)
# print("---------------------------------------------")
# test_corpus,_,_= utils.get_train_dataset(test_data)
# print(test_corpus)
# test_as_tokenized_string=feature_extractor.list_of_lists(test_corpus)

In [ ]:
# print(test_as_tokenized_string)

In [ ]:
# class TestLargeDataset(Dataset):
#     def __init__(self, data):
#         self.data = data 
    
#     def __len__(self):
#         return len(self.data)
    
#     def __getitem__(self, idx):
#         return self.data[idx]

# def test_collate_fn(batch):
#     sequences = batch
#     #I believe we can transform words into embeddings here
#     embeddings=[]
#     for seq in sequences:
#         print(seq)
#         x=[]
#         for token in seq:
#             x.append(emb_model.wv[token])
#         embeddings.append(x)
#     sequences=embeddings
#     sequences = [torch.tensor(seq) for seq in sequences]
#     padded_sequences = pad_sequence(sequences, batch_first=True)
#     lengths = torch.tensor([len(seq) for seq in sequences], dtype=torch.long)
#     return padded_sequences, lengths


# test_as_tokenized_string
# dataset = TestLargeDataset(test_as_tokenized_string)
# dataloader = DataLoader(dataset, batch_size=1, collate_fn=test_collate_fn, shuffle=False, num_workers=0)

# for padded_sequences, lengths in dataloader:
#     print("-------------------------------")
#     padded_sequences=padded_sequences.to(device)
#     lengths=lengths.to(device)
#     outputs = model(padded_sequences, lengths)
#     entity_to_num = {"I_NUMBER": 0, "I_SIZE": 1, "I_TOPPING": 2, "I_STYLE": 3, "I_DRINKTYPE": 4, "I_CONTAINERTYPE": 5, "I_VOLUME": 6, "I_QUANTITY": 7, "B_NUMBER": 8, "B_SIZE": 9, "B_TOPPING": 10, "B_STYLE": 11, "B_DRINKTYPE": 12, "B_CONTAINERTYPE": 13, "B_VOLUME": 14, "B_QUANTITY": 15, "I_NOT_TOPPING": 16, "B_NOT_TOPPING": 17, "NONE": 18}
#     for i, out in enumerate(outputs[0]):
#         num = torch.argmax(out).int().item()
#         for key, value in entity_to_num.items():
#             if value == num:
#                 print(key)
#                 break


    
# #  entity_to_num = {"I_NUMBER": 0, "I_SIZE": 1, "I_TOPPING": 2, "I_STYLE": 3, "I_DRINKTYPE": 4, "I_CONTAINERTYPE": 5, "I_VOLUME": 6, "I_QUANTITY": 7, "B_NUMBER": 8, "B_SIZE": 9, "B_TOPPING": 10, "B_STYLE": 11, "B_DRINKTYPE": 12, "B_CONTAINERTYPE": 13, "B_VOLUME": 14, "B_QUANTITY": 15, "I_NOT_TOPPING": 16, "B_NOT_TOPPING": 17, "NONE": 18}


In [ ]:
    # entity_to_num = {"I_NUMBER": 0, "I_SIZE": 1, "I_TOPPING": 2, "I_STYLE": 3, "I_DRINKTYPE": 4, "I_CONTAINERTYPE": 5, "I_VOLUME": 6, "I_QUANTITY": 7, "B_NUMBER": 8, "B_SIZE": 9, "B_TOPPING": 10, "B_STYLE": 11, "B_DRINKTYPE": 12, "B_CONTAINERTYPE": 13, "B_VOLUME": 14, "B_QUANTITY": 15, "I_NOT_TOPPING": 16, "B_NOT_TOPPING": 17, "NONE": 18}


In [ ]:
dev_data=utils.read_data("../data/fixed_PIZZA_dev.json")
dev_corpus, dev_top = utils.get_dev_dataset(dev_data)
gold_dev_labels,dev_as_tokenized_string=utils.label_complete_dev(dev_corpus, dev_top)

class TestLargeDataset(Dataset):
    def __init__(self, data):
        self.data = data 
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

def test_collate_fn(batch):
    sequences = batch
    #I believe we can transform words into embeddings here
    embeddings=[]
    for seq in sequences:
        print(seq)
        x=[]
        for token in seq:
            x.append(emb_model.wv[token])
        embeddings.append(x)
    sequences=embeddings
    sequences = [torch.tensor(seq) for seq in sequences]
    padded_sequences = pad_sequence(sequences, batch_first=True)
    lengths = torch.tensor([len(seq) for seq in sequences], dtype=torch.long)
    return padded_sequences, lengths

dev_dataset = TestLargeDataset(dev_as_tokenized_string)
dataloader = DataLoader(dev_dataset, batch_size=1, collate_fn=test_collate_fn, shuffle=False, num_workers=0)
model_output=[]
for padded_sequences, lengths in dataloader:
    print("-------------------------------")
    labels = []
    padded_sequences=padded_sequences.to(device)
    lengths=lengths.to(device)
    outputs = model(padded_sequences, lengths)
    entity_to_num = {"I_NUMBER": 0, "I_SIZE": 1, "I_TOPPING": 2, "I_STYLE": 3, "I_DRINKTYPE": 4, "I_CONTAINERTYPE": 5, "I_VOLUME": 6, "I_QUANTITY": 7, "B_NUMBER": 8, "B_SIZE": 9, "B_TOPPING": 10, "B_STYLE": 11, "B_DRINKTYPE": 12, "B_CONTAINERTYPE": 13, "B_VOLUME": 14, "B_QUANTITY": 15, "I_NOT_TOPPING": 16, "B_NOT_TOPPING": 17,"I_NOT_STYLE": 18, "B_NOT_STYLE": 19, "B_NOT_QUANTITY": 20, "I_NOT_QUANTITY": 21, "NONE": 22}
    for i, out in enumerate(outputs[0]):
        num = torch.argmax(out).int().item()
        labels.append(num)
    model_output.append(labels)

confusion_matrix, accuracy=utils.calc_accuracy(model_output, gold_dev_labels)
print(confusion_matrix)
print(accuracy)

In [ ]:
# import pandas as pd
# import os
# import opendatasets as od
# # ! pip install opendatasets
# # Assign the Kaggle data set URL into variable
# dataset = 'https://www.kaggle.com/datasets/ghaithqoudeiamti/pizza-dataset?select=food_review2.txt'
# # Using opendatasets let's download the data sets
# od.download(dataset)

In [ ]:
# model = pickle.load(open('../models/model_1m.pk1' , 'rb'))
# model = torch.load('../models/model_1m.pk1', map_location=torch.device('cpu'))
model = torch.load('../models/model_1m.pk1', map_location=torch.device('cpu'))

